In [1]:
#Import dependencies
from pathlib import Path
import pandas as pd
import scipy.stats as stats

In [2]:
#Open filtered CSV file
traffic_injuries = Path("Resources/filtered-road-traffic-injuries-2002-2010.csv")
df = pd.read_csv(traffic_injuries)

# Read in the data. 
df

/var/folders/gj/34pvzc9x2x98g0vpglqmrsz80000gn/T/ipykernel_12022/1203060863.py:3: DtypeWarning: Columns (0,5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(traffic_injuries)


,Unnamed: 0,ind_definition,reportyear,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,...,avmttotal,avmtrate,LL95CI_avmtrate,UL95CI_avmtrate,avmtrate_se,avmtrate_rse,CA_decile_avmt,CA_RR_avmtrate,Unnamed: 29,Unnamed: 30
0,NaN,Annual number of fatal and severe road traffic...,2002,CO,6059.0,Orange,Orange,6059.0,Southern California,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Annual number of fatal and severe road traffic...,2002,CO,6059.0,Orange,Orange,6059.0,Southern California,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Annual number of fatal and severe road traffic...,2002,CO,6059.0,Orange,Orange,6059.0,Southern California,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Annual number of fatal and severe road traffic...,2002,CO,6059.0,Orange,Orange,6059.0,Southern California,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Annual number of fatal and severe road traffic...,2002,CO,6059.0,Orange,Orange,6059.0,Southern California,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
448946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
448947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
448948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#clean data to remove null values and columns 
clean_df = df.drop(columns = ["ind_definition","version"])
clean_df = clean_df.drop(clean_df[clean_df['ind_id'] == "END OF TABLE"].index)
clean_df = clean_df.drop(columns = ["ind_id", 'LL95CI_avmtrate', 'UL95CI_avmtrate',
       'avmtrate_se', 'avmtrate_rse', 'CA_decile_avmt', 'CA_RR_avmtrate',
       'groupquarters','LL95CI_poprate', 'UL95CI_poprate',
       'poprate_se', 'poprate_rse', 'CA_decile_pop', 'CA_RR_poprate',
       'avmttotal', 'avmtrate'])
clean_df

KeyError: "['ind_id', 'groupquarters', 'LL95CI_poprate', 'UL95CI_poprate', 'poprate_se', 'poprate_rse', 'CA_decile_pop', 'CA_RR_poprate'] not found in axis"

In [4]:
clean_df.isnull().sum()

NameError: name 'clean_df' is not defined

In [12]:
#make modes of transport float with 1.0 representing whether all modes, bus, bike, etc. for each row
mode_float = pd.get_dummies(clean_df["mode"], dtype=float)
mode_float = mode_float.drop(columns = ["ND"])
mode_float

,All modes,Bicyclist,Bus,Car/Pickup,Motorcycle,Pedestrian,Truck,Vehicles
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
448944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
448945,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
448946,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
448947,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [15]:
#make float for type of severity of accident. Killed or injured
severity_float = pd.get_dummies(clean_df["severity"], dtype=float)
#drop ND column added
severity_float = severity_float.drop(columns = ["ND"])
#make float list to concatenate since you cannot merge
floats = [mode_float, severity_float]
severity_float

,Killed,Severe Injury
0,1.0,0.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0
...,...,...
448944,0.0,1.0
448945,0.0,1.0
448946,0.0,1.0
448947,0.0,1.0


In [23]:
#concatentate outer to get new df with 1.0 value denoting mode and injury
merged_mode_injury =  pd.concat(floats, axis=1, join="outer")
merged_mode_injury.head()

,All modes,Bicyclist,Bus,Car/Pickup,Motorcycle,Pedestrian,Truck,Vehicles,Killed,Severe Injury
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [26]:
#Create new dataframe, this is better than clean to be referenced when making graphs of data
new_df = pd.DataFrame(
    {"reportyear" : clean_df["reportyear"],
    "county_name" : clean_df["county_name"],
    "region_name" : clean_df["region_name"],
    "region_code" : clean_df["region_code"],
    "All modes" : merged_mode_injury["All modes"],
    "Bicyclist" : merged_mode_injury["Bicyclist"],
    "Bus": merged_mode_injury["Bus"],
    "Car/Pickup" : merged_mode_injury["Car/Pickup"],
    "Motorcycle" : merged_mode_injury["Motorcycle"],
    "Pedestrian" : merged_mode_injury["Pedestrian"],
    "Truck" : merged_mode_injury["Truck"],
    "Vehicles" : merged_mode_injury["Vehicles"],
    "Killed" : merged_mode_injury["Killed"],
    "Severe Injury" : merged_mode_injury["Severe Injury"],
    "injuries" : clean_df["injuries"],
    "totalpop" : clean_df["totalpop"],
    "poprate" : clean_df["poprate"]
 }
 )

new_df

,reportyear,county_name,region_name,region_code,All modes,Bicyclist,Bus,Car/Pickup,Motorcycle,Pedestrian,Truck,Vehicles,Killed,Severe Injury,injuries,totalpop,poprate
0,2002,Orange,Southern California,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,193.0,2914663.0,6.62
1,2002,Orange,Southern California,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,721.0,2914663.0,24.74
2,2002,Orange,Southern California,14.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,2914663.0,0.24
3,2002,Orange,Southern California,14.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,49.0,2914663.0,1.68
4,2002,Orange,Southern California,14.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2914663.0,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448944,2010,Nevada,Northeast Sierra,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,NaN,NaN
448945,2010,Nevada,Northeast Sierra,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,16180.0,12.36
448946,2010,Nevada,Northeast Sierra,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,16180.0,6.18
448947,2010,Nevada,Northeast Sierra,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,16180.0,6.18


In [ ]:
#separate data frames by year
years = new_df["reportyear"].unique()

In [ ]:
#Make graph comparing every mode of travel by every type of injury
#make graph breaking down how many were killed or severly injured by type of travel, maybe calculate it percent to make pie graph or do a bar chart
#compare total accidents per region
#graphs comparing accidnets and miles traveled? Or region and miles traveled? Might help with linear regression later. See comment in box with correlation matrix to see how to use miles traveled
#Do these for every year
#Do these summing up all the years, maybe make dataframes specific to a year
#any other ideas?

In [83]:
#need to do linear regression model of graphs. What does the data tell us? is there trend in injury over year? by region? By mode of transport?
#What about by miles traveled?(this will be found all the way back in beginnning under "df["avmtrate"] look at the excel dictionary I think we can still use this. If used be sure to remove rows are same as clean_df or new_df) 


# Assuming df is your DataFrame
correlation_matrix = clean_df[["geotypevalue","county_fips", "region_code", "injuries", "totalpop", "poprate"]].describe()

#I dont think this is the correlation we are looking for in regard to final project presentation. I think we should make a dataframe that
#is summed up and more about accident per region, modes of transport, and miles traveled.

# Display the correlation matrix
print(correlation_matrix)
#clean_df.info()

       geotypevalue    county_fips   region_code       injuries      totalpop  \
count  4.489480e+05  445371.000000  94066.000000  447565.000000  1.133240e+05   
mean   4.816763e+09    6055.795018      8.762082       6.797583  2.552469e+05   
std    3.169910e+09      27.275320      4.892614     122.888407  1.868938e+06   
min    1.000000e+00    6001.000000      1.000000       0.200000  0.000000e+00   
25%    6.011000e+09    6037.000000      4.000000       0.670000  4.424000e+03   
50%    6.037573e+09    6059.000000     10.000000       1.000000  9.174000e+03   
75%    6.071009e+09    6073.000000     14.000000       2.000000  6.256300e+04   
max    1.000000e+11    6115.000000     14.000000   13578.000000  3.725396e+07   

             poprate  
count  111634.000000  
mean       27.683685  
std       244.278075  
min         0.000000  
25%         3.790000  
50%         7.930000  
75%        19.090000  
max     26000.000000  
